## Data Transformation Trial-runs

In [1]:
import os 

In [2]:
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\NLP-Text-Summerizer")

## Entity Trial-runs  

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

## ConfigurationManager Trial-runs 

In [4]:
#importing all project paths and modules necessary for project configurations 
from textSummerizer.constants import  *
from textSummerizer.utils.common import read_yaml, create_directories

In [5]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation 
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path, 
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

## Components Trial-runs 

In [6]:
import os 
from textSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\USER\anaconda3\envs\NLPTextSummerizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-26 14:23:44,963: INFO: config: PyTorch version 2.7.0 available.]


In [7]:
class DataTransformation: 
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings =self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
            return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'], 
            'labels': target_encodings['input_ids']
        }

    def convert(self): 
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_example_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))    

## Pipeline trial-runs

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config =config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-05-26 14:23:46,201: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-05-26 14:23:46,233: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-05-26 14:23:46,243: INFO: common: Created directory at: artifacts]
[2025-05-26 14:23:46,243: INFO: common: Created directory at: artifacts/data_transformation]


c:\Users\USER\anaconda3\envs\NLPTextSummerizer\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\USER\anaconda3\envs\NLPTextSummerizer\lib\site-packages\transformers\tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 22657.39 examples/s]
